## bitslicing

the warp cipher [detail](warp.ipynb).

In [2]:
def integer_to_bitslicing(b0):
    b0_b = bin(b0)[2:].zfill(128)
    R0_b = ""
    R1_b = ""
    R2_b = ""
    R3_b = ""
    for i in range(0, len(b0_b), 4):
        R0_b += b0_b[i]
        R1_b += b0_b[i + 1]
        R2_b += b0_b[i + 2]
        R3_b += b0_b[i + 3]

    return int(R0_b, 2), int(R1_b, 2), int(R2_b, 2), int(R3_b, 2)


def bitslicing_to_integer(R0, R1, R2, R3):
    R0_b = bin(R0)[2:].zfill(32)
    R1_b = bin(R1)[2:].zfill(32)
    R2_b = bin(R2)[2:].zfill(32)
    R3_b = bin(R3)[2:].zfill(32)
    b0_b = ""
    for i in range(0, len(R0_b)):
        b0_b += R0_b[i]
        b0_b += R1_b[i]
        b0_b += R2_b[i]
        b0_b += R3_b[i]
    return int(b0_b, 2)

b0 = 0x0123456789abcdef0123456789abcdef
r0, r1, r2, r3 = integer_to_bitslicing(b0)
b = bitslicing_to_integer(r0, r1, r2, r3)
assert b == b0

## sbox 

$$\begin{align}
    T_{0}  & =  X_{0} \land X_{1}  & T_{1}  & =  X_{0} \land X_{2}  & T_{2}  & =  X_{2} \lor T_{0}   \nonumber       \\
    T_{3}  & =  X_{0} \lor X_{3}   & T_{4}  & =  X_{3} \lor X_{1}   & T_{5}  & =  T_{2} \land T_{3} \nonumber       \\
    T_{6}  & =  T_{2} \oplus T_{0}  & T_{7}  & =  T_{4} \land X_{0}  & T_{8}  & =  X_{1} \lor T_{7}    \nonumber      \\
    T_{9}  & =  T_{7} \oplus X_{1}  & T_{10} & =  \sim T_{0}         & T_{11} & =  T_{4} \oplus T_{0}        \tag{1}          \\
    T_{12} & =  T_{7} \land X_{3}   & T_{13} & =  T_{9} \land T_{3}  & T_{14} & =  T_{6} \lor T_{13}       \nonumber \\
    T_{15} & =  T_{8} \land X_{2}   & T_{16} & = \sim T_{5}          & T_{17} & = \sim T_{3}       \nonumber \\
    T_{18} & =  T_{11} \oplus T_{12} & T_{19} & =  T_{15} \lor T_{17} & Y_{0}  & = T_{16}                 \nonumber    \\
    Y_{1}  & = T_{19}               & Y_{2}  & = T_{18}              & Y_{3}  & = T_{14} \nonumber
\end{align}
$$

In [3]:
S = [0xC, 0xA, 0xD, 0x3, 0xE, 0xB, 0xF, 0x7, 0x8, 0x9, 0x1, 0x5, 0x0, 0x2, 0x4, 0x6]
def sbox(R0, R1, R2, R3):
    r0, r1, r2, r3 = 0, 0, 0, 0
    T = [0] * 20
    # T_{0} =  X_{0} \land X_{1} 
    T[0] = R0 & R1
    # T_{1} =  X_{0} \land X_{2}
    T[1] = R0 & R2
    # T_{2} =  X_{2} \lor T_{0}
    T[2] = R2 | T[0]
    # T_{3} =  X_{0} \lor X_{3} 
    T[3] = R0 | R3
    # T_{4} =  X_{3} \lor X_{1} 
    T[4] = R3 | R1
    # T_{5} =  T_{2} \land T_{3}
    T[5] = T[2] & T[3]
    # T_{6} =  T_{2} \oplus T_{0}
    T[6] = T[2] ^ T[0]
    # T_{7} =  T_{4} \land X_{0} 
    T[7] = T[4] & R0
    # T_{8} =  X_{1} \lor T_{7} 
    T[8] = R1 | T[7]
    # T_{9} =  T_{7} \oplus X_{1}
    T[9] = T[7] ^ R1
    # T_{10} = \sim T_{0}
    T[10] = T[0] ^ 0xFFFF_FFFF
    # T_{11} =  T_{4} \oplus T_{0} 
    T[11] = T[4] ^ T[0]
    # T_{12} =  T_{7} \land X_{3}
    T[12] = T[7] & R3
    # T_{13} =  T_{9} \land T_{3}
    T[13] = T[9] & T[3]
    # T_{14} =  T_{6} \lor T_{13} 
    T[14] = T[6] | T[13]
    # T_{15} =  T_{8} \land X_{2} 
    T[15] = T[8] & R2
    # T_{16} = \sim T_{5} 
    T[16] = T[5] ^ 0xFFFF_FFFF
    # T_{17} = \sim T_{3} 
    T[17] = T[3] ^ 0xFFFF_FFFF
    # T_{18} =  T_{11} \oplus T_{12} 
    T[18] = T[11] ^ T[12]
    # T_{19} =  T_{15} \lor T_{17} 
    T[19] = T[15] | T[17]
    # Y_{0} = T_{16}
    r0 = T[16]
    # Y_{1} = T_{19}
    r1 = T[19]
    # Y_{2} = T_{18}
    r2 = T[18]
    # Y_{3} = T_{14}
    r3 = T[14]

    return r0, r1, r2, r3

b0 = 0x0123456789abcdef0123456789abcdef
r0, r1, r2, r3 = integer_to_bitslicing(b0)
print(f'r0 = {hex(r0)} r1 = {hex(r1)} r2 = {hex(r2)} r3 = {hex(r3)}')
r0, r1, r2, r3 = sbox(r0, r1, r2, r3)
print(f'r0 = {hex(r0)} r1 = {hex(r1)} r2 = {hex(r2)} r3 = {hex(r3)}')
b = bitslicing_to_integer(r0, r1, r2, r3)
b_expected = 0xcad3ebf789150246cad3ebf789150246
assert b == b_expected

b1 = 0x00020406080a0c0e00020406080a0c0e
r0, r1, r2, r3 = integer_to_bitslicing(b1)

r0 = 0xff00ff r1 = 0xf0f0f0f r2 = 0x33333333 r3 = 0x55555555
r0 = 0xeec0eec0 r1 = 0xab13ab13 r2 = 0x5f055f05 r3 = 0x37703770


## permutation 

In [4]:
def right_cyclic_shift(n, bits, shift_amount):
    return ((n >> shift_amount) | (n << (bits - shift_amount))) & ((1 << bits) - 1)


def left_cyclic_shift(n, bits, shift_amount):
    return ((n << shift_amount) | (n >> (bits - shift_amount))) & ((1 << bits) - 1)

In [5]:
# the pi is shuffled version of 0..31, need change to permutation.
pi = [
    31,
    6,
    29,
    14,
    1,
    12,
    21,
    8,
    27,
    2,
    3,
    0,
    25,
    4,
    23,
    10,
    15,
    22,
    13,
    30,
    17,
    28,
    5,
    24,
    11,
    18,
    19,
    16,
    9,
    20,
    7,
    26,
]

pi_p = [pi.index(i) for i in range(32)]
print(f"pi_p = {pi_p}")


def permutation_bitslicing(R0, R1, R2, R3, p):
    shift = [i - p[i] if i - p[i] >= 0 else i - p[i] + 32 for i in range(32)]
    # print(f"shift = {shift}")
    r0 = 0
    r1 = 0
    r2 = 0
    r3 = 0
    for i in range(32):
        r0 |= (right_cyclic_shift(R0, 32, shift[i])) & (1 << ((31 - i)))
        r1 |= (right_cyclic_shift(R1, 32, shift[i])) & (1 << ((31 - i)))
        r2 |= (right_cyclic_shift(R2, 32, shift[i])) & (1 << ((31 - i)))
        r3 |= (right_cyclic_shift(R3, 32, shift[i])) & (1 << ((31 - i)))
    return r0, r1, r2, r3


b0 = 0x0123456789ABCDEF0123456789ABCDEF
r0, r1, r2, r3 = integer_to_bitslicing(b0)
print(f"r0 = {hex(r0)} r1 = {hex(r1)} r2 = {hex(r2)} r3 = {hex(r3)}")
r0, r1, r2, r3 = permutation_bitslicing(r0, r1, r2, r3, pi_p)
print(f"r0 = {hex(r0)} r1 = {hex(r1)} r2 = {hex(r2)} r3 = {hex(r3)}")
b = bitslicing_to_integer(r0, r1, r2, r3)
print(f"b = {hex(b)}")
b2 = [
    0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
]
b2 = [b2[pi_p[i]] for i in range(32)]
b3 = 0
for i in range(32):
    b3 = (b3 << 4) | b2[i]
assert b == b3

pi_p = [11, 4, 9, 10, 13, 22, 1, 30, 7, 28, 15, 24, 5, 18, 3, 16, 27, 20, 25, 26, 29, 6, 17, 14, 23, 12, 31, 8, 21, 2, 19, 0]
r0 = 0xff00ff r1 = 0xf0f0f0f r2 = 0x33333333 r3 = 0x55555555
shift = [21, 29, 25, 25, 23, 15, 5, 9, 1, 13, 27, 19, 7, 27, 11, 31, 21, 29, 25, 25, 23, 15, 5, 9, 1, 13, 27, 19, 7, 27, 11, 31]
r0 = 0xb970b970 r1 = 0x4de84de8 r2 = 0x95a695a6 r3 = 0xaaaaaaaa
b = 0xb49ad61e7cf85230b49ad61e7cf85230


## add round constant

In [6]:
RC0s = [
    0x0,
    0x0,
    0x1,
    0x3,
    0x7,
    0xF,
    0xF,
    0xF,
    0xE,
    0xD,
    0xA,
    0x5,
    0xA,
    0x5,
    0xB,
    0x6,
    0xC,
    0x9,
    0x3,
    0x6,
    0xD,
    0xB,
    0x7,
    0xE,
    0xD,
    0xB,
    0x6,
    0xD,
    0xA,
    0x4,
    0x9,
    0x2,
    0x4,
    0x9,
    0x3,
    0x7,
    0xE,
    0xC,
    0x8,
    0x1,
    0x2,
]

RC1s = [
    0x4,
    0xC,
    0xC,
    0xC,
    0xC,
    0xC,
    0x8,
    0x4,
    0x8,
    0x4,
    0x8,
    0x4,
    0xC,
    0x8,
    0x0,
    0x4,
    0xC,
    0x8,
    0x4,
    0xC,
    0xC,
    0x8,
    0x4,
    0xC,
    0x8,
    0x4,
    0x8,
    0x0,
    0x4,
    0x8,
    0x0,
    0x4,
    0xC,
    0xC,
    0x8,
    0x0,
    0x0,
    0x4,
    0x8,
    0x4,
    0xC,
]

# X[1] = X[1] ^ RC0s[r]
# X[3] = X[3] ^ RC1s[r]
def add_round_constant(R0, R1, R2, R3, RC0, RC1):
    r0 = R0 ^ (((RC0 >> 3) & 1) << 30) ^ (((RC1 >> 3) & 1) << (28))
    r1 = R1 ^ (((RC0 >> 2) & 1) << 30) ^ (((RC1 >> 2) & 1) << (28))
    r2 = R2 ^ (((RC0 >> 1) & 1) << 30) ^ (((RC1 >> 1) & 1) << (28))
    r3 = R3 ^ (((RC0 >> 0) & 1) << 30) ^ (((RC1 >> 0) & 1) << (28))
    return r0, r1, r2, r3

b0 = 0x0123456789ABCDEF0123456789ABCDEF
r0, r1, r2, r3 = integer_to_bitslicing(b0)
r0, r1, r2, r3 = add_round_constant(r0, r1, r2, r3, 1, 3)
b = bitslicing_to_integer(r0, r1, r2, r3)
b_expected = 0x0020456789ABCDEF0123456789ABCDEF
assert b == b_expected

## add round key

In [7]:
# use trick to temp use. 64-bit store in 128-bit

# encrypt

$$K=0x0123456789ABCDEFFEDCBA9876543210$$
$$M=0x0123456789ABCDEFFEDCBA9876543210$$
$$C=0x24CE0A8EFD9F32DE529D5FDF45703A8D$$

In [8]:
M = 0x0123456789ABCDEFFEDCBA9876543210
K0 = 0x000102030405060708090A0B0C0D0E0F
K1 = 0x0F0E0D0C0B0A09080706050403020100
C = 0x24CE0A8EFD9F32DE529D5FDF45703A8D

In [9]:
X = M
k0r0, k0r1, k0r2, k0r3 = integer_to_bitslicing(K0)
k1r0, k1r1, k1r2, k1r3 = integer_to_bitslicing(K1)
r0, r1, r2, r3 = integer_to_bitslicing(X)
ks = []
print(f"M = {hex(r0)} {hex(r1)} {hex(r2)} {hex(r3)}")
# head round 40
for r in range(40):
    # print(f"round {r}")
    # print(f"x = {hex(bitslicing_to_integer(r0, r1, r2, r3))}")
    print(f"r{r} start = {hex(r0)} {hex(r1)} {hex(r2)} {hex(r3)}")
    # even branch
    t0, t1, t2, t3 = r0 & 0xAAAAAAAA, r1 & 0xAAAAAAAA, r2 & 0xAAAAAAAA, r3 & 0xAAAAAAAA
    t0, t1, t2, t3 = sbox(t0, t1, t2, t3)
    # to odd branch
    t0, t1, t2, t3 = (
        (t0 >> 1) & 0x55555555,
        (t1 >> 1) & 0x55555555,
        (t2 >> 1) & 0x55555555,
        (t3 >> 1) & 0x55555555,
    )
    # odd branch add the even branch passed sbox
    r0, r1, r2, r3 = r0 ^ t0, r1 ^ t1, r2 ^ t2, r3 ^ t3
    # print(f"s = {hex(bitslicing_to_integer(r0, r1, r2, r3))}")
    print(f"r{r} f = {hex(r0)} {hex(r1)} {hex(r2)} {hex(r3)}")
    # add round key
    if (r) % 2 == 0:
        # this k* is the key and the round constant
        k0 = k0r0 ^ (((RC0s[r] >> 3) & 1) << 30) ^ (((RC1s[r] >> 3) & 1) << (28))
        k1 = k0r1 ^ (((RC0s[r] >> 2) & 1) << 30) ^ (((RC1s[r] >> 2) & 1) << (28))
        k2 = k0r2 ^ (((RC0s[r] >> 1) & 1) << 30) ^ (((RC1s[r] >> 1) & 1) << (28))
        k3 = k0r3 ^ (((RC0s[r] >> 0) & 1) << 30) ^ (((RC1s[r] >> 0) & 1) << (28))
        r0, r1, r2, r3 = (
            r0 ^ k0,
            r1 ^ k1,
            r2 ^ k2,
            r3 ^ k3,
        )
        ks.append(k0)
        ks.append(k1)
        ks.append(k2)
        ks.append(k3)
        print(f"r{r} k = {hex(k0)}, {hex(k1)}, {hex(k2)}, {hex(k3)},")
    else:
        k0 = k1r0 ^ (((RC0s[r] >> 3) & 1) << 30) ^ (((RC1s[r] >> 3) & 1) << (28))
        k1 = k1r1 ^ (((RC0s[r] >> 2) & 1) << 30) ^ (((RC1s[r] >> 2) & 1) << (28))
        k2 = k1r2 ^ (((RC0s[r] >> 1) & 1) << 30) ^ (((RC1s[r] >> 1) & 1) << (28))
        k3 = k1r3 ^ (((RC0s[r] >> 0) & 1) << 30) ^ (((RC1s[r] >> 0) & 1) << (28))
        r0, r1, r2, r3 = (
            r0 ^ k0,
            r1 ^ k1,
            r2 ^ k2,
            r3 ^ k3,
        )
        ks.append(k0)
        ks.append(k1)
        ks.append(k2)
        ks.append(k3)
        print(f"r{r} k = {hex(k0)}, {hex(k1)}, {hex(k2)}, {hex(k3)},")
    # print(f"k = {hex(bitslicing_to_integer(r0, r1, r2, r3))}")
    # add round constant
    # r0, r1, r2, r3 = add_round_constant(r0, r1, r2, r3, RC0s[r], RC1s[r])
    # print(f"c = {hex(bitslicing_to_integer(r0, r1, r2, r3))}")
    # permutation
    print(f"r{r} p_b = {hex(r0)} {hex(r1)} {hex(r2)} {hex(r3)}")
    r0, r1, r2, r3 = permutation_bitslicing(r0, r1, r2, r3, pi_p)
    print(f"r{r} p = {hex(r0)} {hex(r1)} {hex(r2)} {hex(r3)}")

# tail round 41
# even branch
t0, t1, t2, t3 = r0 & 0xAAAAAAAA, r1 & 0xAAAAAAAA, r2 & 0xAAAAAAAA, r3 & 0xAAAAAAAA
t0, t1, t2, t3 = sbox(t0, t1, t2, t3)
# to odd branch
t0, t1, t2, t3 = (
    (t0 >> 1) & 0x55555555,
    (t1 >> 1) & 0x55555555,
    (t2 >> 1) & 0x55555555,
    (t3 >> 1) & 0x55555555,
)
# odd branch add the even branch passed sbox
r0, r1, r2, r3 = r0 ^ t0, r1 ^ t1, r2 ^ t2, r3 ^ t3

k0 = k0r0 ^ (((RC0s[40] >> 3) & 1) << 30) ^ (((RC1s[40] >> 3) & 1) << (28))
k1 = k0r1 ^ (((RC0s[40] >> 2) & 1) << 30) ^ (((RC1s[40] >> 2) & 1) << (28))
k2 = k0r2 ^ (((RC0s[40] >> 1) & 1) << 30) ^ (((RC1s[40] >> 1) & 1) << (28))
k3 = k0r3 ^ (((RC0s[40] >> 0) & 1) << 30) ^ (((RC1s[40] >> 0) & 1) << (28))

# add round key
r0, r1, r2, r3 = (
    r0 ^ k0,
    r1 ^ k1,
    r2 ^ k2,
    r3 ^ k3,
)
ks.append(k0)
ks.append(k1)
ks.append(k2)
ks.append(k3)
print(f"r{r} last = {hex(r0)} {hex(r1)} {hex(r2)} {hex(r3)}")

# add round constant
# r0, r1, r2, r3 = add_round_constant(r0, r1, r2, r3, RC0s[40], RC1s[40])
c = bitslicing_to_integer(r0, r1, r2, r3)
assert c == C

M = 0xffff00 0xf0ff0f0 0x3333cccc 0x5555aaaa
r0 start = 0xffff00 0xf0ff0f0 0x3333cccc 0x5555aaaa
r0 f = 0x55bffe11 0x5a0eb4b0 0x36339c99 0x4445affe
r0 k = 0x5555, 0x10550055, 0x5050505, 0x11111111,
r0 p_b = 0x55bfab44 0x4a5bb4e5 0x3336999c 0x5554beef
shift = [21, 29, 25, 25, 23, 15, 5, 9, 1, 13, 27, 19, 7, 27, 11, 31, 21, 29, 25, 25, 23, 15, 5, 9, 1, 13, 27, 19, 7, 27, 11, 31]
r0 p = 0x9eaf69d0 0xe2353d6a 0x98d3cd86 0xafdf782a
r1 start = 0x9eaf69d0 0xe2353d6a 0x98d3cd86 0xafdf782a
r1 f = 0x8eab28c4 0xa674786b 0xc9d3d993 0xbb9a387f
r1 k = 0x45550000, 0x45005500, 0x50505050, 0x44444444,
r1 p_b = 0xcbfe28c4 0xe3742d6b 0x998389c3 0xffde7c3b
shift = [21, 29, 25, 25, 23, 15, 5, 9, 1, 13, 27, 19, 7, 27, 11, 31, 21, 29, 25, 25, 23, 15, 5, 9, 1, 13, 27, 19, 7, 27, 11, 31]
r1 p = 0xfa946d51 0xbbc474ad 0x41b361b1 0xebced77f
r2 start = 0xfa946d51 0xbbc474ad 0x41b361b1 0xebced77f
r2 f = 0xfa802814 0xbb9464fd 0x14f620e4 0xebdfd72b
r2 k = 0x10005555, 0x10550055, 0x5050505, 0x51111111,
r2 p_b = 0xea80

In [41]:
# Convert each number to hex format and store in a new list
hex_ks = [hex(number) for number in ks]

# Join the hex strings with commas
formatted_hex_ks = ','.join(hex_ks)

print(formatted_hex_ks)

0x5555,0x10550055,0x5050505,0x11111111,0x45550000,0x45005500,0x50505050,0x44444444,0x10005555,0x10550055,0x5050505,0x51111111,0x45550000,0x45005500,0x10505050,0x4444444,0x10005555,0x50550055,0x45050505,0x51111111,0x5550000,0x5005500,0x10505050,0x4444444,0x50005555,0x40550055,0x45050505,0x51111111,0x15550000,0x5005500,0x10505050,0x4444444,0x50005555,0x40550055,0x45050505,0x11111111,0x15550000,0x5005500,0x50505050,0x4444444,0x50005555,0x550055,0x45050505,0x11111111,0x55550000,0x5005500,0x50505050,0x4444444,0x50005555,0x10550055,0x45050505,0x11111111,0x45550000,0x15005500,0x50505050,0x4444444,0x40005555,0x550055,0x45050505,0x51111111,0x55550000,0x5005500,0x10505050,0x44444444,0x50005555,0x50550055,0x5050505,0x11111111,0x5550000,0x55005500,0x50505050,0x4444444,0x5555,0x10550055,0x45050505,0x51111111,0x45550000,0x5005500,0x10505050,0x44444444,0x50005555,0x50550055,0x5050505,0x51111111,0x5550000,0x55005500,0x10505050,0x4444444,0x5555,0x50550055,0x45050505,0x51111111,0x5550000,0x5005500,0x105